In [5]:
%config Completer.use_jedi = False

import pandas as pd
from sqlalchemy import create_engine

## Extract ExcelFiles into DataFrames

In [35]:
athletes_file = "./Resources/Athletes.xlsx"
athletes_df = pd.read_excel(athletes_file, sheet_name='Details', engine='openpyxl')
athletes_df.head()

Name     NOC           Discipline
0    AALERUD Katrine  Norway         Cycling Road
1        ABAD Nestor   Spain  Artistic Gymnastics
2  ABAGNALE Giovanni   Italy               Rowing
3     ABALDE Alberto   Spain           Basketball
4      ABALDE Tamara   Spain           Basketball

In [36]:
coaches_file = "./Resources/Coaches.xlsx"
coaches_df = pd.read_excel(coaches_file, sheet_name='Details', engine='openpyxl')
coaches_df.head()

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Name            NOC  Discipline Event
0  ABDELMAGID Wael          Egypt    Football   NaN
1        ABE Junya          Japan  Volleyball   NaN
2    ABE Katsuhiko          Japan  Basketball   NaN
3     ADAMA Cherif  Côte d'Ivoire    Football   NaN
4       AGEBA Yuya          Japan  Volleyball   NaN

In [37]:
teams_file = "./Resources/Teams.xlsx"
teams_df = pd.read_excel(teams_file, sheet_name='Details', engine='openpyxl')
teams_df.head()

Name      Discipline                         NOC  Event
0  Belgium  3x3 Basketball                     Belgium    Men
1    China  3x3 Basketball  People's Republic of China    Men
2    China  3x3 Basketball  People's Republic of China  Women
3   France  3x3 Basketball                      France  Women
4    Italy  3x3 Basketball                       Italy  Women

## Columns to transform from data source files

In [38]:
country_cols = ['NOC']
discipline_cols = ['Discipline']
athletes_cols = ['Name', 'NOC', 'Discipline']
coaches_cols = ['Name', 'NOC', 'Discipline']

## Transform country DataFrame

In [90]:
country_transformed = athletes_df[country_cols].copy().drop_duplicates().reset_index().rename(columns={'index': 'country_id','NOC': 'country_name'})
country_transformed.head()

country_id country_name
0           0       Norway
1           1        Spain
2           2        Italy
3           5       France
4           6        Chile

## Transform discipline DataFrame

In [89]:
discipline_transformed = athletes_df[discipline_cols].copy().drop_duplicates().reset_index().rename(columns={'index': 'discipline_id','Discipline': 'discipline'})
discipline_transformed.head()

discipline_id           discipline
0              0         Cycling Road
1              1  Artistic Gymnastics
2              2               Rowing
3              3           Basketball
4              5             Handball

## Transform Atheletes & Coaches DataFrame

In [92]:
atheletes_transformed = athletes_df[athletes_cols].copy().drop_duplicates().reset_index().rename(columns={'index': 'athelete_id','Name': 'athelete_name', 'NOC': 'country_name', 'Discipline': 'discipline'})
atheletes_transformed.head()

athelete_id      athelete_name country_name           discipline
0            0    AALERUD Katrine       Norway         Cycling Road
1            1        ABAD Nestor        Spain  Artistic Gymnastics
2            2  ABAGNALE Giovanni        Italy               Rowing
3            3     ABALDE Alberto        Spain           Basketball
4            4      ABALDE Tamara        Spain           Basketball

In [94]:
coaches_transformed = coaches_df[coaches_cols].copy().drop_duplicates().reset_index().rename(columns={'index': 'coache_id','NOC': 'country_name','Name': 'coache_name', 'Discipline':'discipline'})
coaches_transformed.head()

coache_id      coache_name   country_name  discipline
0          0  ABDELMAGID Wael          Egypt    Football
1          1        ABE Junya          Japan  Volleyball
2          2    ABE Katsuhiko          Japan  Basketball
3          3     ADAMA Cherif  Côte d'Ivoire    Football
4          4       AGEBA Yuya          Japan  Volleyball

In [95]:
coaches_transformed_with_country_id = coaches_transformed.merge(country_transformed, how="inner", on=["country_name"]).rename(columns={'country_id': 'coache_country_id'})
coaches_transformed_with_discipline_id = coaches_transformed_with_country_id.merge(discipline_transformed, how="inner", on=["discipline"]).rename(columns={'discipline_id': 'coache_discipline_id'}).drop(columns=['country_name', 'discipline'])
coaches_transformed_with_discipline_id

coache_id        coache_name  coache_country_id  coache_discipline_id
0            0    ABDELMAGID Wael                 14                    25
1           12          ALY Kamal                 14                    25
2          316      SALAM Mohamed                 14                    25
3          329      SHAWKY Gharib                 14                    25
4          330      SIAM Zaghloul                 14                    25
..         ...                ...                ...                   ...
376        199   KNEZEVIC Nemanja                 38                    87
377        235        MARTIN Paul                 61                    87
378        247    MENTOOR Delaine                 61                    87
379        131  GOJKOVIC Vladimir                446                    87
380        358        TUCAK Ivica               1008                    87

[381 rows x 4 columns]

In [96]:
atheletes_transformed_with_country_id = atheletes_transformed.merge(country_transformed, how="outer", on=["country_name"]).rename(columns={'country_id': 'athelete_country_id'})
atheletes_transformed_with_discipline_id = atheletes_transformed_with_country_id.merge(discipline_transformed, how="outer", on=["discipline"]).rename(columns={'discipline_id': 'athelete_discipline_id'}).drop(columns=['country_name', 'discipline'])
atheletes_transformed_with_discipline_id

athelete_id               athelete_name  athelete_country_id  \
0                0             AALERUD Katrine                    0   
1             1153                BORGLI Stine                    0   
2             2979              FOSS Tobias S.                    0   
3             3982            HOELGAARD Markus                    0   
4             4446  JOHANNESSEN Tobias Halland                    0   
...            ...                         ...                  ...   
11079        11045                 ZHU Xueying                  156   
11080         7904            PROSTOROV Mykola                  231   
11081         3701        HANCHAROU Uladzislau                  344   
11082         5621            LITVINOVICH Ivan                  344   
11083         2136           DAVIDSON Madaline                  369   

       athelete_discipline_id  
0                           0  
1                           0  
2                           0  
3                           0  
4                           0  
...                       ...  
11079                      65  
11080                      65  
11081                      65  
11082                      65  
11083                      65  

[11084 rows x 4 columns]

## Create Database Connection

In [101]:
connection_string = "{}:{}@localhost:5432/olympics".format('postgres', 'wns7267')
engine = create_engine(f'postgresql://{connection_string}')

In [124]:
# Confirm tables
engine.table_names()

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  


['countries', 'disciplines', 'atheletes', 'coaches']

## Load DataFrames into Database

In [112]:
try: 
    country_transformed.to_sql(name='countries', con=engine, if_exists='replace', index=False)
except:
    pass

In [113]:
try: 
    discipline_transformed.to_sql(name='disciplines', con=engine, if_exists='replace', index=False)
except:
    pass

In [114]:
try: 
    atheletes_transformed_with_discipline_id.to_sql(name='atheletes', con=engine, if_exists='replace', index=False)
except:
    pass

In [115]:
try:
    coaches_transformed_with_discipline_id.to_sql(name='coaches', con=engine, if_exists='replace', index=False)
except:
    pass

In [120]:
# confirm the insertion
countries=pd.read_sql_query('select * from countries',con=engine)
countries

country_id              country_name
0             0                    Norway
1             1                     Spain
2             2                     Italy
3             5                    France
4             6                     Chile
..          ...                       ...
201        5889                    Tuvalu
202        5944   Virgin Islands, British
203        6190  Central African Republic
204        6887                   Myanmar
205        7259                  Suriname

[206 rows x 2 columns]

In [121]:
disciplines=pd.read_sql_query('select * from disciplines',con=engine)
disciplines

discipline_id             discipline
0               0           Cycling Road
1               1    Artistic Gymnastics
2               2                 Rowing
3               3             Basketball
4               5               Handball
5               7               Swimming
6               8                 Karate
7               9              Wrestling
8              11    Rhythmic Gymnastics
9              12      Baseball/Softball
10             13              Athletics
11             14      Artistic Swimming
12             16                   Judo
13             17               Shooting
14             20           Table Tennis
15             25               Football
16             26              Taekwondo
17             31                Fencing
18             32              Badminton
19             34                 Boxing
20             37          Weightlifting
21             38                Archery
22             48                 Diving
23             53       Beach Volleyball
24             54                Sailing
25             61                 Hockey
26             65  Trampoline Gymnastics
27             67      Marathon Swimming
28             80              Triathlon
29             86           Canoe Slalom
30             87             Water Polo
31            107                Surfing
32            108           Canoe Sprint
33            115     Cycling BMX Racing
34            123           Rugby Sevens
35            179             Volleyball
36            188             Equestrian
37            227                 Tennis
38            331          Cycling Track
39            375                   Golf
40            377          Skateboarding
41            492      Modern Pentathlon
42            596  Cycling Mountain Bike
43            826         3x3 Basketball
44            911  Cycling BMX Freestyle
45           1638         Sport Climbing

In [122]:
atheletes=pd.read_sql_query('select * from atheletes',con=engine)
atheletes

athelete_id               athelete_name  athelete_country_id  \
0                0             AALERUD Katrine                    0   
1             1153                BORGLI Stine                    0   
2             2979              FOSS Tobias S.                    0   
3             3982            HOELGAARD Markus                    0   
4             4446  JOHANNESSEN Tobias Halland                    0   
...            ...                         ...                  ...   
11079        11045                 ZHU Xueying                  156   
11080         7904            PROSTOROV Mykola                  231   
11081         3701        HANCHAROU Uladzislau                  344   
11082         5621            LITVINOVICH Ivan                  344   
11083         2136           DAVIDSON Madaline                  369   

       athelete_discipline_id  
0                           0  
1                           0  
2                           0  
3                           0  
4                           0  
...                       ...  
11079                      65  
11080                      65  
11081                      65  
11082                      65  
11083                      65  

[11084 rows x 4 columns]

In [119]:
coaches=pd.read_sql_query('select * from coaches',con=engine)
coaches

coache_id        coache_name  coache_country_id  coache_discipline_id
0            0    ABDELMAGID Wael                 14                    25
1           12          ALY Kamal                 14                    25
2          316      SALAM Mohamed                 14                    25
3          329      SHAWKY Gharib                 14                    25
4          330      SIAM Zaghloul                 14                    25
..         ...                ...                ...                   ...
376        199   KNEZEVIC Nemanja                 38                    87
377        235        MARTIN Paul                 61                    87
378        247    MENTOOR Delaine                 61                    87
379        131  GOJKOVIC Vladimir                446                    87
380        358        TUCAK Ivica               1008                    87

[381 rows x 4 columns]